# 文本分类

---




In [1]:
import numpy as np

In [2]:
def load_data():
    from sklearn.datasets import fetch_20newsgroups

    categories = ['alt.atheism', 'soc.religion.christian',
                  'comp.graphics', 'sci.med']
    twenty_train = fetch_20newsgroups(subset='train',
                                      categories=categories,
                                      shuffle=True, random_state=68)
    twenty_test = fetch_20newsgroups(subset='test',
                                     categories=categories,
                                     shuffle=True,
                                     random_state=68)
    return twenty_train, twenty_test

# 加载数据集
train, test = load_data()

# from sklearn.feature_extraction.text import CountVectorizer
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(train.data)

# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# X_train = X_train_tfidf.toarray()

In [16]:
from gensim.corpora import Dictionary

corpus = [x.split() for x in train['data']]
dct = Dictionary(corpus)

texts_with_id = [dct.doc2idx(x.split()) for x in train['data']]

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np


class TextCNN(nn.Module):
    def __init__(self, n_class, vocab_size, dim, max_len):
        super(TextCNN, self).__init__()
        self.max_len = max_len
        self.dim = dim
        self.embedding = nn.Embedding(vocab_size, dim)
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5,
                      stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5,
                      stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5,
                      stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5,
                      stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out = nn.Linear(512, n_class)

    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), 1, self.max_len, self.dim)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output


In [6]:
cnn = TextCNN(n_class=4, vocab_size=100000, dim=100, max_len=1000)

In [7]:
LR = 0.001
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
# 损失函数
loss_function = nn.CrossEntropyLoss()

In [11]:
# 训练批次大小
epoch_size = 100
train_x = torch.Tensor(X_train)
train_y = torch.LongTensor(Y_train)
# test_x = torch.LongTensor(X_test)
# test_y = torch.LongTensor(Y_test)
EPOCH = 10

test_epoch_size = 300
for epoch in range(EPOCH):
    for i in range(0, int(len(train_x) / epoch_size)):
        b_x = Variable(torch.LongTensor(train_x[i * epoch_size:i * epoch_size + epoch_size]))
        b_y = Variable(torch.LongTensor((train_y[i * epoch_size:i * epoch_size + epoch_size])))
        output = cnn(b_x)
        loss = loss_function(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(str(i))
        print(loss)
        pred_y = torch.max(output, 1)[1].data.squeeze()
        acc = (b_y == pred_y)
        acc = acc.numpy().sum()
        accuracy = acc / (b_y.size(0))

#     acc_all = 0
#     for j in range(0, (int)(len(test_x) / test_epoch_size)):
#         b_x = Variable(torch.LongTensor(test_x[j * test_epoch_size:j * test_epoch_size + test_epoch_size]))
#         b_y = Variable(torch.LongTensor((test_y[j * test_epoch_size:j * test_epoch_size + test_epoch_size])))
#         test_output = cnn(b_x)
#         pred_y = torch.max(test_output, 1)[1].data.squeeze()
#         # print(pred_y)
#         # print(test_y)
#         acc = (pred_y == b_y)
#         acc = acc.numpy().sum()
#         print("acc " + str(acc / b_y.size(0)))
#         acc_all = acc_all + acc

#     accuracy = acc_all / (test_y.size(0))
#     print("epoch " + str(epoch) + " step " + str(i) + " " + "acc " + str(accuracy))


TypeError: torch.LongTensor constructor received an invalid combination of arguments - got (torch.FloatTensor), but expected one of:
 * no arguments
 * (int ...)
      didn't match because some of the arguments have invalid types: (!torch.FloatTensor!)
 * (torch.LongTensor viewed_tensor)
      didn't match because some of the arguments have invalid types: (!torch.FloatTensor!)
 * (torch.Size size)
      didn't match because some of the arguments have invalid types: (!torch.FloatTensor!)
 * (torch.LongStorage data)
      didn't match because some of the arguments have invalid types: (!torch.FloatTensor!)
 * (Sequence data)
      didn't match because some of the arguments have invalid types: (!torch.FloatTensor!)
